In [1]:
import pandas as pd
import os
import numpy as np

In [6]:
df_info=pd.read_csv(r'/home/payal/Desktop/cricket /odi_info.csv')

In [7]:
read_files=df_info['match-id']

In [9]:
# delete_files=[]
# for index in read_files.index:
#     if '(' in read_files[index]:
#         delete_files.append(index)
# read_files=read_files.drop(delete_files)

In [10]:
files=[file for dirpath,directory,file in os.walk('/home/payal/all_csv/')][0]

In [11]:
match_data=pd.DataFrame(data=None)
odi_scorecard=pd.DataFrame(data=None)
odi_info=pd.DataFrame(data=None)

In [12]:
def rename_date_umpire(index_list):
    n=0
    for i in range(index_list.__len__()):
        if str.lower(index_list[i]).strip()=='date':
            index_list[i]+='_'+str(n)
            n+=1
    n=0
    for i in range(index_list.__len__()):
        if str.lower(index_list[i]).strip()=='umpire':
            index_list[i]+='_'+str(n)
            n+=1
    return index_list

In [13]:
def append_file(temp_df,temp_info_df):
    global odi_scorecard
    global odi_info
    odi_scorecard=odi_scorecard.append(temp_df,ignore_index=True)
    odi_info=odi_info.append(temp_info_df,ignore_index=True)

In [270]:
def get_extras_type(match_data):
    list_extras=[]
    for index,row in match_data.iterrows():
        ov=str(row['over'])
        if '.' in ov:
            ov=str(row['over']).split('.')
            ball_no=int(ov[1])
            over_no=int(ov[0])
        else:
            continue
        if row['extras']!=0:
            if row['runs']!=0:
                match_data.loc[index,'extras_type']='w'
            list_extras.append(index)
        if ball_no>6:
            if len(list_extras)>0:
                match_data.loc[list_extras.pop(-1),'extras_type']='w'
    for i in list_extras:
        match_data.loc[i,'extras_type']='b'
    return match_data

In [271]:
def prepare_scorecard(match_data):
    match_data=get_extras_type(match_data)
    over=[1.1,1.2,1.3,1.4,1.5,1.6,2.1,2.2,2.3,2.4,2.5,2.6]
    over_stats=['match_id','team_name','innings','over_ball','name','ball_score','wicket','wicket_by','batsman_out','extras']
    over_data={key:{key_type:0 for key_type in over_stats} for key in over}
    for index, row in match_data.iterrows():
        ov=str(row['over'])
        if '.' in ov:
            ov=ov.split('.')
            ball_no=int(ov[1])
            over_no=int(ov[0])
        else:
            continue
        if over_no==0:
            ball_no=(row['innings']*1.0)+(ball_no/10)
            if over_data.get(ball_no,-1)==-1:
                over_data[ball_no]={key:0 for key in over_stats}
            over_data[ball_no]['name']=row['striker']
            over_data[ball_no]['innings']=row['innings']
            over_data[ball_no]['match_id']=row['file_no']
            over_data[ball_no]['over_ball']=ball_no
            over_data[ball_no]['ball_score']=row['runs']
            over_data[ball_no]['wicket']=row['out']
            over_data[ball_no]['team_name']=row['batting-team']
            if not str(row['out'])=='nan':
                over_data[ball_no]['wicket_by']=row['bowler']
            over_data[ball_no]['batsman_out']=row['out-player']
            over_data[ball_no]['extras']=row['extras']
#     remove balls which are extras
    over_data=[value for key, value in over_data.items()]
    scorecard=pd.DataFrame(data=over_data)
    scorecard=scorecard[over_stats]
    return scorecard
    

In [272]:
i=0
for file in read_files[:]:
    print(i,'--',file)
    i+=1
    count=0
    df_index=[]
    df_row=[]
    add=r'/home/payal/all_csv/'+file+'.csv'
#     add=r'/gdrive/My Drive/all_csv/'+file
    df_index=['file_no']
    file_no=file.split('.')[0]
    df_row=[file_no]
    with open(add) as f:
        new_f=f.readlines()
        for line in new_f:
            if 'version' in line:
                count+=1
            elif 'info' in line:
                count+=1
                line=line.strip().split(',')
                df_index.append(line[1])
                df_row.append(line[2])
            else:
                df_index=rename_date_umpire(df_index)
                df_dic=dict(zip(df_index,df_row),index=[0])
                temp_info_df=pd.DataFrame(df_dic)
#                 df_info=df_info.append(temp_info_df,ignore_index=True)
                # gender=df_info['gender'].iloc[0]
                # gender=str.lower(gender.strip())
                break         
    temp_df=pd.read_csv(add,skiprows=count,names=[0,'innings','over','batting-team','striker','non-striker','bowler','runs','extras','out','out-player'])
    temp_df=temp_df.drop([0],axis=1)
    temp_df['file_no']=[file_no]*(temp_df.shape[0])
#     temp_df['over_no']=temp_df['ot(rowver'].apply(lambda x: str(x).split('.')[0])
#     temp_df=temp_df[temp_df['over_no']=='0']
    temp_sc=prepare_scorecard(temp_df)
#     append_file(temp_df,gender,type_game) 
    append_file(temp_sc,temp_info_df)
    

0 -- 1000887
1 -- 1000889
2 -- 1000891
3 -- 1000893
4 -- 1000895
5 -- 1001371
6 -- 1001373
7 -- 1001375
8 -- 1003273
9 -- 1004253
10 -- 1004255
11 -- 1004261
12 -- 1004269
13 -- 1004283
14 -- 1004285
15 -- 1007649
16 -- 1007651
17 -- 1007653
18 -- 1014845
19 -- 1014847
20 -- 1019973
21 -- 1019975
22 -- 1019977
23 -- 1020013
24 -- 1020017
25 -- 1020031
26 -- 1020033
27 -- 1020035
28 -- 1020037
29 -- 1020039
30 -- 1022347
31 -- 1022349
32 -- 1022351
33 -- 1022353
34 -- 1022355
35 -- 1022357
36 -- 1022359
37 -- 1022361
38 -- 1022363
39 -- 1022365
40 -- 1022367
41 -- 1022369
42 -- 1022371
43 -- 1022373
44 -- 1022375
45 -- 1026847
46 -- 1026849
47 -- 1026851
48 -- 1027315
49 -- 1027317
50 -- 1027319
51 -- 1029001
52 -- 1029003
53 -- 1029819
54 -- 1029821
55 -- 1029823
56 -- 1030219
57 -- 1030221
58 -- 1030223
59 -- 1030225
60 -- 1030227
61 -- 1031425
62 -- 1031427
63 -- 1031429
64 -- 1033363
65 -- 1033365
66 -- 1033367
67 -- 1033369
68 -- 1033371
69 -- 1033707
70 -- 1034819
71 -- 1034821
72

573 -- 313992
574 -- 313993
575 -- 313994
576 -- 317578
577 -- 317579
578 -- 317581
579 -- 319134
580 -- 319135
581 -- 319136
582 -- 319143
583 -- 319144
584 -- 319145
585 -- 319146
586 -- 319147
587 -- 323949
588 -- 323950
589 -- 323951
590 -- 325569
591 -- 325579
592 -- 325580
593 -- 325581
594 -- 325800
595 -- 325801
596 -- 325802
597 -- 325803
598 -- 325804
599 -- 335346
600 -- 335347
601 -- 335348
602 -- 335349
603 -- 335350
604 -- 335351
605 -- 335352
606 -- 335353
607 -- 335354
608 -- 335355
609 -- 335356
610 -- 335357
611 -- 335358
612 -- 336201
613 -- 336202
614 -- 336204
615 -- 343732
616 -- 343733
617 -- 343734
618 -- 343736
619 -- 343759
620 -- 343760
621 -- 343761
622 -- 343762
623 -- 343763
624 -- 345468
625 -- 345469
626 -- 345471
627 -- 350043
628 -- 350044
629 -- 350045
630 -- 350046
631 -- 350047
632 -- 350048
633 -- 350049
634 -- 350348
635 -- 350477
636 -- 350478
637 -- 350479
638 -- 350480
639 -- 350481
640 -- 351684
641 -- 351685
642 -- 351686
643 -- 351687
644 --

1149 -- 582190
1150 -- 584546
1151 -- 584547
1152 -- 584549
1153 -- 584928
1154 -- 587471
1155 -- 587472
1156 -- 587473
1157 -- 587474
1158 -- 587475
1159 -- 589308
1160 -- 589309
1161 -- 589310
1162 -- 592266
1163 -- 593723
1164 -- 593724
1165 -- 593725
1166 -- 593983
1167 -- 593985
1168 -- 594891
1169 -- 594894
1170 -- 594897
1171 -- 594901
1172 -- 594904
1173 -- 594907
1174 -- 594909
1175 -- 594912
1176 -- 594914
1177 -- 594915
1178 -- 597923
1179 -- 597924
1180 -- 597925
1181 -- 597926
1182 -- 597927
1183 -- 597928
1184 -- 597929
1185 -- 601077
1186 -- 601612
1187 -- 601613
1188 -- 602474
1189 -- 602476
1190 -- 603239
1191 -- 603240
1192 -- 603241
1193 -- 623573
1194 -- 623574
1195 -- 623575
1196 -- 627006
1197 -- 627007
1198 -- 628081
1199 -- 628083
1200 -- 628086
1201 -- 628090
1202 -- 628092
1203 -- 628093
1204 -- 628095
1205 -- 628096
1206 -- 631138
1207 -- 635653
1208 -- 635654
1209 -- 635655
1210 -- 635656
1211 -- 635657
1212 -- 636159
1213 -- 636160
1214 -- 636161
1215 -- 63

1697 -- 995457
1698 -- 995459
1699 -- 995461
1700 -- 995463
1701 -- 995465
1702 -- 997961
1703 -- 997965
1704 -- 997967
1705 -- 997993
1706 -- 997995


In [273]:
odi_scorecard.to_csv(r'/home/payal/Desktop/odi_temp.csv',index=False)